# DS44000 Homework 1

Chris Dilger

Code available on GitHub: https://github.com/cdilga/DS4400/blob/master/Homework%201.ipynb

--------------------------------------------------------------------------------

## Problem 1


In [142]:
%config IPCompleter.greedy=True
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def cols(data, width = 30):
    """Formats list like objects into columns"""
    return "".join(str(word).ljust(30) for word in data)

                   
# Load the file
housing = pd.read_csv("data/train.csv", 
                      usecols=lambda x: x in ["price",
                                              "bedrooms",
                                              "bathrooms",
                                              "sqft_living",
                                              "sqft_lot",
                                              "floors",
                                              "waterfront",
                                              "view",
                                              "condition",
                                              "grade",
                                              "sqft_above",
                                              "sqft_basement",
                                              "yr_built",
                                              "yr_renovated",
                                              "lat",
                                              "long",
                                              "sqft_living15",
                                              "sqft_lot15"])

housing_test = pd.read_csv("data/test.csv", 
                      usecols=lambda x: x in ["price",
                                              "bedrooms",
                                              "bathrooms",
                                              "sqft_living",
                                              "sqft_lot",
                                              "floors",
                                              "waterfront",
                                              "view",
                                              "condition",
                                              "grade",
                                              "sqft_above",
                                              "sqft_basement",
                                              "yr_built",
                                              "yr_renovated",
                                              "lat",
                                              "long",
                                              "sqft_living15",
                                              "sqft_lot15"])

description = housing.describe().loc[["mean", "std", "min", "max"]]
description = description.T
description['var'] = description['std'].apply(lambda x: x**2)
description


,mean,std,min,max,var
price,520414.834000,339488.477270,80000.0000,3.075000e+06,1.152524e+11
bedrooms,3.349000,0.852012,0.0000,7.000000e+00,7.259249e-01
bathrooms,2.045750,0.721623,0.0000,5.000000e+00,5.207402e-01
sqft_living,2051.196000,887.929222,380.0000,6.070000e+03,7.884183e+05
sqft_lot,14702.085000,28961.030775,649.0000,3.153740e+05,8.387413e+08
floors,1.446500,0.517354,1.0000,3.500000e+00,2.676554e-01
waterfront,0.008000,0.089129,0.0000,1.000000e+00,7.943944e-03
view,0.237000,0.765125,0.0000,4.000000e+00,5.854164e-01
condition,3.464000,0.689332,1.0000,5.000000e+00,4.751792e-01
grade,7.606000,1.160220,4.0000,1.200000e+01,1.346110e+00


In [143]:
def cov(x, y):
    """Calculate covariance given the feature x and response y"""
    total = 0
    x_m = x.mean()
    y_m = y.mean()
    for i in range(len(x)):
        total += (x.loc[i]-x_m)*(y.loc[i]-y_m)
    return total/(len(x)-1)

def cor(x, y):
    """Calculate the correlation coefficient"""
    return cov(x, y)/(x.std()*y.std())

for feature in [x for x in housing if x != "price"]:
    print(cols([feature, 
                cor(housing["price"], housing[feature]),
                housing["price"].corr(housing[feature])
               ]))
    #print()

bedrooms                      0.3070584002104321            0.3070584002104319            
bathrooms                     0.48715729789866974           0.4871572978986769            
sqft_living                   0.7047757101823997            0.7047757101824055            
sqft_lot                      0.14664482983805222           0.14664482983805197           
floors                        0.2399348472639126            0.2399348472639161            
waterfront                    0.31714301382621846           0.3171430138262226            
view                          0.44531642588846515           0.4453164258884609            
condition                     0.07396055390188862           0.07396055390188938           
grade                         0.647349055785599             0.6473490557856066            
sqft_above                    0.5824071469756538            0.5824071469756582            
sqft_basement                 0.3673649191324655            0.36736491913246594           

#### Discussion of Problem 1.a
Suprisingly every feature is positively correlated with price, which is suprising. `yr_built` and `yr_renovated` are weakly correlated, as were `long` (longditude) and `sqft_lot15`. 


## Problem 2 [Linear regression]

[(a)] Use an existing package to train a linear regression model on the training set. Report the coefficients of the linear regression models and the 3 metrics of interest: MSE, RSS, and $R^2$.


In [145]:
#use scikit.learn

#Get MSE RSS and R^2

#All predictors predicting 'price'

feature_words = [x for x in housing if x != "price"]
features = housing.loc[:, feature_words]
labels = housing.loc[:,'price']


from sklearn import linear_model

reg = linear_model.LinearRegression()
reg.fit(features, labels)

from IPython.display import display, Math
display(Math("R^2 = {0:.4f}".format(reg.score(features, labels))))

from sklearn import metrics
mse = metrics.mean_squared_error(labels, reg.predict(features))
display(Math("MSE={0:.0f}".format(mse)))

rss = mse*len(labels)
display(Math("RSS={0:.0f}".format(rss)))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Problem 3 [Closed form solution of linear regression]


In this problem, you will implement your own linear regression model, using the closed-form solution we derived in class. You will also compare your model with the one trained with the package.


1. Implement simple linear regression and train a model for one feature (sqft\_living) using the training set. Write code to predict a response for a new single-dimensional data point in the testing set.




In [215]:
class Lin:
    """Fit a linear model to some datapoints in 2 dimensions"""
    def __init__(self):
        self.theta = np.ones(2)
        self._rss = 0
        self.n = 0
        self._tss = 0
        
    def fit(self, X, y): 
        """Takes X Feature numpy 1d array, y Label numpy 1d array"""
        
        self.n = X.shape[0]
        
        x_m = np.mean(X)
        y_m = np.mean(y)
        
        # for each training data point in x, and corresponding label y
        # calculate 2 separate values, using this formula and a mean of each
        a = b = 0
        for i in range(self.n):
            diffx = X[i]-x_m
            a += (diffx)*(y[i]-y_m)
            b += (diffx)**2
            
        self.theta[1] = a/b
        self.theta[0] = y_m-self.theta[1]*x_m
        
        for i in range(self.n):
            self._rss += (y[i] - self.predict(X[i]))**2
            self._tss += (y[i] - y_m)**2
        
        return self
    
    def rss(self):
        return self._rss
    
    def mse(self):
        return (self._rss / self.n)
    
    def r2(self):
        return 1-(self._rss/self._tss)
    
    def coef(self):
        """Return the coefficients vector"""
        return self.theta
    
    def predict(self, x):
        return self.theta[0] + x*self.theta[1]

mod = Lin()

housing_training_features
mod.fit(housing.loc[:,'sqft_living'], housing.loc[:,'price'])

thetas = mod.coef()
display(Math("\\theta_0 = {0:.4f}, \\theta_1 = {1:.4f}".format(thetas[0], thetas[1])))

predict = housing_test.loc[1,'sqft_living']
model = mod.predict(predict)
actual = housing_test.loc[1,'price']

display(Math("\\text{Predicted}=" + 
             str(round(model, 2)) + 
             " \\space ft^2\\\\ \\text{Actual}= " + 
             str(round(actual, 2)) + 
             "\\space ft^2"
            ))

display(Math("RSS = {:.4f} \\\\ MSE = {:.4f} \\\\ R^2 = {:.4f}".format(mod.rss(), mod.mse(), mod.r2())))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

2. Implement multiple linear regression using matrix operations and train a model on the training set. Write code to predict a response for a new multi-dimensional data point in the testing set.



In [346]:
class Multi_lin:
    # has a coeficients property
    # has a fit method
    
    def __init__(self):
        self.theta = np.ones((1,1))
        self._rss = 0
        self.n = 0
        self._tss = 0
        
    def fit(self, X, y): 
        #matrix operation for the mean of each feature of x?
        self.n = X.shape[0]
        y_m = y.mean()
        #use theta
        self.theta = np.linalg.pinv(X.T.dot(X)).dot(X.T).dot(y)
        self._calc_rss(X, y, y_m)
        return self
    
    def coef(self):
        return self.theta
    
    def _calc_rss(self, X, y, y_m):
        for i in range(self.n):
            self._rss += (y[i] - self.predict(X[i]))**2
            self._tss += (y[i] - y_m)**2
            
    def mse(self):
        return (self._rss / self.n)
    def rse(self):
        raise Exception('Not implemented')
    def rss(self):
        return self._rss
    def r2(self):
        raise Exception('Not implemented')
    def add(x, y, self):
        raise Exception('Not implemented')
    def predict(self, x):
        return self.theta.T.dot(x)
    
    
mult = Multi_lin()

#This should be 4 features
mult.fit(np.random.random((40, 4)), np.random.random((40,1)))
mult.predict(np.random.random(4))
#print(mult.coef())


housing_training_features = housing.copy().drop("price", axis=1)

housing_training_labels = housing.loc[:,"price"]
housing_testing_features = housing_test.copy().drop("price", axis=1)
housing_testing_labels = housing_test.loc[:,"price"]

mult.fit(np.array(housing_training_features), housing_training_labels)
model = mult.predict(np.array(housing_testing_features))
#actual = housing_testing_labels

ValueError: shapes (17,) and (1000,17) not aligned: 17 (dim 0) != 1000 (dim 0)

3. Compare the models given by your implementation with those trained in Problem 2 by the R or Python packages. Report the MSE, RSE, and $R^2$ metrics for the models you implemented. Compare the coefficients output by your model with the ones computed by the package.


## Problem 5 [Ridge Regression]
Derivation of the closed form solution of Ridge Regression

Begin with the definition of the loss function:

$$J(\theta)=\frac{1}{2}\sum_{i=1}^n{[h_\theta(x^{(i)})-y^{(i)}}]^2+\frac{1}{2}\lambda \sum_{j=1}^d{\theta_j^2}$$
$$\begin{align}
y=&x\\
=&0
\end{align}
$$
where $d$ is the dimensions of the training data\\
$h_\theta(x)=\sum_{m=0}^d{\theta_mx_m}$\\
$x^{(i)}$ denotes the $i$th datapoint of the training dataset

We resolve to find the minima of a convex function defined by the cost function $J(\theta)$

Generally the approach will be to 
\begin{enumerate}
    \item Find the gradient of the function $J(\theta)$
    
    
\end{enumerate}


Derivation of the closed form solution of Ridge Regression

Begin with the definition of the loss function:\\
$$J(\theta)=\frac{1}{2}\sum_{i=1}^n{[h_\theta(x^{(i)})-y^{(i)}}]^2+\frac{1}{2}\lambda \sum_{j=1}^d{\theta_j^2}$$
where $d$ is the dimensions of the training data

$h_\theta(x)=\sum_{m=0}^d{\theta_mx_m}$


$x^{(i)}$ denotes the $i$th datapoint of the training dataset

We resolve to find the minima of a convex function defined by the cost function $J(\theta)$

Generally the approach will be to:
\begin{enumerate}
    \item Find the gradient of the function $J(\theta)$
    \item Set the gradient to zero
    \item Solve for the $\theta$
\end{enumerate}
$$J(\theta)=\frac{1}{2}\sum_{i=1}^n{[h_\theta(x^{(i)})-y^{(i)}]}^2+\frac{1}{2}\lambda \sum_{j=1}^d{\theta_j^2}$$

$$$$


$$
\frac{\partial J\theta)}{\partial\theta_l}\\
    = & \frac{\partial}{\partial \theta_l}\frac{1}{2}
    \left[
        \sum_{i=1}^n
        {
        \left[
            \theta_0+\theta_1x_1^{(i)}+\theta_2x_2^{(i)}+\dots+\theta_l x_l^{(i)}
        \right]
        }
        -y^{(i)}
    \right]^2
    +\frac{\partial}{\partial \theta_l}
    \left[
        \frac{\lambda}{2}
        \sum_{j=1}^d{\theta_j}
    \right]\\
$$

For readability we now take the left half of the sum to compute
$$
    \frac{\partial}{\partial \theta_l}\frac{1}{2}
    \left[
        \sum_{i=1}^n
        {
        \left[
            \theta_0+\theta_1x_1^{(i)}+\theta_2x_2^{(i)}+\dots+\theta_l x_l^{(i)}
        \right]
        }
        -y^{(i)}
    \right]^2\\
    = \frac{1}{2}\left[\sum_{i=1}^n
    \left[
        \frac{\partial}{\partial \theta_l}\left(\theta_0\right)
        +\frac{\partial}{\partial \theta_l}\left(\theta_1x_1^{(i)}\right)
        +\frac{\partial}{\partial \theta_l}\left(\theta_2x_2^{(i)}\right)
        + \dots
        +\frac{\partial}{\partial \theta_l}\left(\theta_l x_l^{(i)}\right)
        + \dots
        +\frac{\partial}{\partial \theta_l}\left(\theta_k x_k^{(i)}\right)
    \right]-\frac{\partial}{\partial \theta_l}y^{(i)}
    \right]\\
    \cdot 2 \cdot \left[ \sum_{i=0}^n{
    \left[
        \sum_{k=0}^d
        {
        \left[
            \theta_k x_k^{(i)}
        \right] -y^{(i)}
        }
    \right]
    }\right]\\
    = x_l^{(i)}\sum_{i=1}^n
    \left[
        \sum_{k=0}^d
        {\left[
            \left(\theta_kx_k^{(i)}\right) - y^{(i)}
        \right]}
    \right]
$$
For $0<l\leq k$

Now to reintroduce the right hand sum
$$
        \frac{\partial}{\partial \theta_l}
    \left[
        \frac{\lambda}{2}
        \sum_{j=1}^d{\theta_j}
    \right]\\
    = \frac{\lambda}{2}\cdot 2 \theta_L\\
    = \lambda\theta_l
$$
Given that $l\in[1,d]$}

Combining equations again
$$
          \frac{\partial J(\theta)}{\partial}
 x_l^{(i)}\sum_{i=1}^n
    \left[
        \sum_{k=0}^d
        {\left[
            \left(\theta_k x_k^{(i)}\right) - y^{(i)}
        \right]}
    \right]
    +
    \lambda\theta_l =0\\
    = x_l^{(i)}\sum_{i=1}^n
    \left[
        h_\theta(x^{(i)}) - y^{(i)}
    \right]
    +
    \lambda\theta_l =0\\
    = \sum_{i=1}^n
    x_l^{(i)}
    {\left[
        \sum_{k=0}^{l-1}
        {\left[
            \left(\theta_k x_k^{(i)}\right) - y^{(i)}
        \right]}
        +
        \left(\theta_lx+l^{(i)}\right)
        +
        \sum_{k=l+1}^d
        {\left[
            \left(\theta_k x_k^{(i)}\right) - y^{(i)}
        \right]}
    \right]}
$$